# Use Case 1: Dwelling and Cafe dataset

In [1]:
import requests
import pandas as pd
import os
import numpy as np

## API Function

In [136]:
def fetch_data(base_url, dataset, api_key, num_records, max_offset,where, offset):
    all_records =[]
    #Maximum number of requests

    while True:
        # Maximum limit check
        if offset > max_offset:
            break
        
        # Create API request URL
        
        filters = f'{dataset}/records?&limit={num_records}&offset={offset}&where={where}'
        url = f'{base_url}{filters}&api_key={api_key}'

        try:
            result = requests.get(url, timeout=10)
            result.raise_for_status()
            records = result.json().get('results')
        except requests.exceptions.RequestException as e:
            raise Exception(f'API request failed: {e}')
        
        if records is None:
            break
        
        all_records.extend(records)
        if len(records) < num_records:
            break
        
        # next cycle offset
        offset += num_records
    
    # Dataframe all data
    df = pd.DataFrame(all_records)
    return df


## Creating the DataSet .
#### - API key is obtained from the MOP wensite.
#### - API for dataset is displayed below:
https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/development-activity-monitor/records?limit=20<br>
https://data.melbourne.vic.gov.au/explore/dataset/cafes-and-restaurants-with-seating-capacity/api/?refine.census_year=2022<br>
https://data.melbourne.vic.gov.au/explore/dataset/employment-by-block-by-clue-industry/api/<br>
https://data.melbourne.vic.gov.au/explore/dataset/employment-by-block-by-clue-industry/api/



In [142]:
# PERSONLA KEY: eb67c6d708fd15de910be7c7a1e7c7eac64c7c0ad1da6c6f7a8fe928
API_KEY = 'c3fe52b3c0f0f3a2d07fc8d89192f5bd3e3234c68b2c30adcd8ab5ca'
BASE_URL = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/"
DATASET_DEVELOPMENT = 'development-activity-monitor'
DATASET_CAFE = 'cafes-and-restaurants-with-seating-capacity'
DATASET_BUS = 'business-establishments-per-block-by-anzsic'
NUM_RECORD = 100
MAX_OFFSET_DEV = 1420
MAX_OFFSET_BUS = 12394
MAX_OFFSET_CAF = 10000
OFFSET = 0
# General select and group_by for Business and Dwelling

WHERE_DEV = ''
WHERE_CAFE ='census_year%20%3E%202021'
# Cafe dataset
#WHERE_CAFE ='industry_anzsic4_code%20%3D%204511'
#WHERE_CAFE="industry_anzsic4_code%20%3D%20'4511'%20AND%20census_year%20%3E%202001%20AND%20census_year%20%3C2007"



WHERE_BUS_2011 = 'census_year%20%3E%202010'
WHERE_BUS_2010 ='census_year%20%3C%202011'

#CAFE DATASET 2022: where=census_year%20%3E%202021
#BUSINESS DATASET > 2010: where=census_year%20%3E%202010
#BUSINESS DATASET <2011: where=census_year%20%3C%202011
#/api/explore/v2.1/catalog/datasets/cafes-and-restaurants-with-seating-capacity/
#records?
#CAFE DATASET: select=census_year%2C%20clue_small_area%2Csum(number_of_seats)
#CAFE DATASET: group_by=census_year%2C%20clue_small_area
#&limit=20
#&refine=industry_anzsic4_description%3A%22Cafes%20and%20Restaurants%22

##
#/api/explore/v2.1/catalog/datasets/cafes-and-restaurants-with-seating-capacity/records?
#select=census_year%2C%20clue_small_area%2C%20count(industry_anzsic4_description)
#&where=industry_anzsic4_code%20%3D%204511
#&group_by=census_year%2C%20clue_small_area
#&limit=20

## Creating three new dataset with API call

### Creating a new dataset called df_dev

In [144]:
df_dev = fetch_data(BASE_URL, DATASET_DEVELOPMENT, API_KEY, NUM_RECORD, MAX_OFFSET_DEV, WHERE_DEV, OFFSET)

In [118]:
# Checking the shape of the new dataset to ensure the size is correct as per the MOP website
df_dev.shape

(1422, 42)

### Creating a new datafset called df_cat from the MOP cafe for year 2022

In [145]:
df_caf= fetch_data(BASE_URL, DATASET_CAFE, API_KEY, NUM_RECORD, MAX_OFFSET_CAF, WHERE_CAFE, OFFSET)

In [130]:
# Checking the shape of the new dataset to ensure the size is correct as per the MOP website
df_caf_2007.shape

(3020, 15)

In [131]:
df_caf

,census_year,block_id,property_id,base_property_id,building_address,clue_small_area,trading_name,business_address,industry_anzsic4_code,industry_anzsic4_description,seating_type,number_of_seats,longitude,latitude,location
0,2002,4,104034,104034,207-361 Flinders Street MELBOURNE 3000,Melbourne (CBD),Coffee HQ,"Store 73, 0-0 Flinders Street MELBOURNE 3000",4511,Cafes and Restaurants,Seats - Indoor,32,144.965785,-37.818683,"{'lon': 144.96578524594116, 'lat': -37.8186831..."
1,2002,6,578320,573333,2 Swanston Street MELBOURNE 3000,Melbourne (CBD),Rowlands Catering,"Tenancy 3, 2 Swanston Street MELBOURNE 3000",4511,Cafes and Restaurants,Seats - Indoor,40,144.969984,-37.817764,"{'lon': 144.96998385754014, 'lat': -37.8177638..."
2,2002,12,103955,103955,475-485 Flinders Lane MELBOURNE 3000,Melbourne (CBD),Cafe Rema,"Ground , 485 Flinders Lane MELBOURNE 3000",4511,Cafes and Restaurants,Seats - Indoor,124,144.958110,-37.819370,"{'lon': 144.958109540856, 'lat': -37.819369527..."
3,2002,12,103995,103995,492-500 Flinders Street MELBOURNE 3000,Melbourne (CBD),Miyabi Japanese Restaurant,"Shop 1, 500 Flinders Street MELBOURNE 3000",4511,Cafes and Restaurants,Seats - Indoor,60,144.957833,-37.819994,"{'lon': 144.95783329800847, 'lat': -37.8199943..."
4,2002,12,103997,103997,472-482 Flinders Street MELBOURNE 3000,Melbourne (CBD),Riverside Deli & Coffee House,"Shop 2, Ground , 474 Flinders Street MELBOURNE...",4511,Cafes and Restaurants,Seats - Indoor,54,144.958281,-37.819729,"{'lon': 144.95828060621216, 'lat': -37.8197289..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015,2003,67,105675,105675,113 Lonsdale Street MELBOURNE 3000,Melbourne (CBD),Cafe Centrum,"Ground , 113 Lonsdale Street MELBOURNE 3000",4511,Cafes and Restaurants,Seats - Outdoor,6,144.969020,-37.810567,"{'lon': 144.9690204858293, 'lat': -37.81056668..."
3016,2003,67,105678,105678,123-129 Lonsdale Street MELBOURNE 3000,Melbourne (CBD),Cafe 123,Part 123-129 Lonsdale Street MELBOURNE 3000,4511,Cafes and Restaurants,Seats - Indoor,22,144.968685,-37.810719,"{'lon': 144.96868472948114, 'lat': -37.8107187..."
3017,2003,67,105892,105892,124-126 Little Bourke Street MELBOURNE 3000,Melbourne (CBD),Golden Orchid Restaurant,"Part Gnd & Mezz , 126 Little Bourke Street MEL...",4511,Cafes and Restaurants,Seats - Indoor,80,144.968769,-37.811313,"{'lon': 144.96876936807104, 'lat': -37.8113130..."
3018,2003,67,105893,105893,120-122 Little Bourke Street MELBOURNE 3000,Melbourne (CBD),Empress of China,120-122 Little Bourke Street MELBOURNE 3000,4511,Cafes and Restaurants,Seats - Indoor,110,144.968919,-37.811317,"{'lon': 144.96891850692202, 'lat': -37.8113168..."


### Creating a new dataset called df_bus from the MOP website<br>
- Need to breakdown the dataset into two parts due to the size of the dataset. 

In [146]:
df_2011 = fetch_data(BASE_URL, DATASET_BUS, API_KEY, NUM_RECORD, MAX_OFFSET_BUS, WHERE_BUS_2011, OFFSET)

Exception: API request failed: 429 Client Error: Too Many Requests for url: https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/business-establishments-per-block-by-anzsic/records?&limit=100&offset=0&where=census_year%20%3E%202010&api_key=c3fe52b3c0f0f3a2d07fc8d89192f5bd3e3234c68b2c30adcd8ab5ca

In [35]:
df_2011.shape

(7279, 23)

In [37]:
df_2010 = fetch_data(BASE_URL, DATASET_BUS, API_KEY, NUM_RECORD, MAX_OFFSET_BUS,WHERE_BUS_2010, OFFSET)

In [39]:
df_2010.shape

(5115, 23)

In [40]:
# Joining the two datasets together
df_bus = pd.concat([df_2011,df_2010])

In [41]:
# Checking the new dataset, expecting the shape to be (12,394, 23)
df_bus.shape

(12394, 23)

### Saving the dataset to drive for future use.

In [43]:
df_dev.to_csv('Dwelling.csv',index = False)

In [44]:
df_caf.to_csv('Cafe.csv',index = False)

In [45]:
df_bus.to_csv('Business.csv',index = False)